In [ ]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath("./")))
from src.tools.base import Tool
from typing import Any, Callable, Type
from inspect import signature
from pydantic import create_model, BaseModel
from pydantic.fields import FieldInfo

In [2]:
# create a dynamic schema for the tool
def _create_function_schema(fn: Callable, name: str) -> Type[BaseModel]:
    """Create a function schema from the fn"""
    fields = {}
    params = signature(fn).parameters
    for param_name in params:
        param_type = params[param_name].annotation
        param_default = params[param_name].default

        if param_type is params[param_name].empty:
            param_type = Any

        if param_default is params[param_name].empty:
            # Required field
            fields[param_name] = (param_type, FieldInfo())
        elif isinstance(param_default, FieldInfo):
            # Field with pydantic.Field as default value
            fields[param_name] = (param_type, param_default)
        else:
            fields[param_name] = (param_type, FieldInfo(default=param_default))

    return create_model(name, **fields)

In [3]:
def add(a: int, b: int) -> int:
    """
    Adds two input numbers
    """
    return a + b

def sub(a: int, b: int) -> int:
    """
    Subtracts two numbers
    """
    return a - b

In [4]:
add_tool = Tool.from_function(function=add)
sub_tool = Tool.from_function(function=sub)

tools = [add_tool, sub_tool]

tool_dict = dict(map(lambda x: (x.tool_name, x), tools))

print(tool_dict)

{'add': Tool(func=<function add at 0x108338d60>, tool_name='add', tool_description='add(a: int, b: int) -> int\n\n    Adds two input numbers\n    ', tool_schema=<class 'src.tools.base.add'>, return_direct=True), 'sub': Tool(func=<function sub at 0x108338e00>, tool_name='sub', tool_description='sub(a: int, b: int) -> int\n\n    Subtracts two numbers\n    ', tool_schema=<class 'src.tools.base.sub'>, return_direct=True)}


In [5]:
add_tool.to_openai_tool()

{'name': 'add',
 'description': 'add(a: int, b: int) -> int\n\n    Adds two functions\n    ',
 'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
   'b': {'title': 'B', 'type': 'integer'}},
  'required': ['a', 'b'],
  'type': 'object'}}